In [1]:
from nanomesh.generator import Generator
import SimpleITK as sitk
from nanomesh.utils import show_slice

import numpy as np

import math

In [10]:
gen = Generator(680, math.sqrt(2)*680, 0.24*680)

# Possible rotation/transformation of the coordinate system
theta = math.pi * 1/180
c = math.cos(theta)
s = math.sin(theta)
trans = np.array([
    [ c, 0, s],
    [ 0, 1, 0],
    [-s, 0, c]
])

vol = gen.generate([100,100,100], [10,10,10], transform=trans)
# Convert to ITK image

im = sitk.GetImageFromArray(vol)
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=49, description='z', max=99), Output()), _dom_classes=('widget-interact'…